#### Imports

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#### Data

In [10]:
base_dir = "data"
train_dir = os.path.join(base_dir, 'train-cat-rabbit')
test_dir = os.path.join(base_dir, 'test-images')
valid_dir = os.path.join(base_dir, 'val-cat-rabbit')

#### Técnicas de Escalamiento y Preprocesado

In [11]:
train_datagen = ImageDataGenerator(
#Técnicas de preprocesamiento de datos para mejorar la categorización del modelo
				rescale = 1./255, 
				rotation_range = 20, 
				width_shift_range = 0.1, 
    			height_shift_range=0.1,
				zoom_range = 0.15,
				horizontal_flip = True,
				brightness_range=[0.5, 1.2] 
    )

validation_datagen = ImageDataGenerator(1./255)  # Escala los píxeles de las imágenes de validación al rango [0, 1]
test_datagen = ImageDataGenerator(1./255)        # Escala los píxeles de las imágenes de prueba al rango [0, 1]


In [ ]:
#Medida de las imágenes
images_size = (150, 150)

train_generator = train_datagen.flow_from_directory(
							train_dir,
							target_size = images_size, # Tamaño al que se redimensionan las imágenes
							batch_size = 10,  # Tamaño del lote de imágenes por batch
							class_mode ='binary', # Tipo de etiquetas: 'binary' porque hay dos clases (gato y conejo)
       						shuffle = True 
							)




plt.figure()
f, axarr = plt.subplots(1, 5, figsize=(30, 8))

for i in range(5) :
  	axarr[i].imshow(train_generator[0][0][0])

In [ ]:
# Visualizar imágenes y las etiquetas despúes de escalamiento y preprocesado
images, labels = train_generator[0]
class_names = list(train_generator.class_indices.keys())

print(images.shape)
print(labels)

plt.figure()
f, axarr = plt.subplots(1, images.shape[0], figsize=(30, 7))

for i in range(images.shape[0]):
    axarr[i].imshow(images[i])
    axarr[i].set_title(class_names[int(labels[i])], fontsize=15)


#### Modelo

In [ ]:
# Crear el modelo CNN

model = Sequential([
    
    # Capa 1: Convolucional con 32 filtros de 3x3, función de activación ReLU, y tamaño de entrada 150x150x3 
    # Capa de MaxPooling para redicir las dimensiones de la salida
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    
    # Capa 2: Convolucional con 64 filtros de 3x3, función de activación ReLU 
    # Capa de MaxPooling para redicir las dimensiones de la salida    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    #Capa 3: Convolucional con 128 filtros de 3x3, función de activación ReLU
    #Capa de MaxPooling para redicir las dimensiones de la salida
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    #Aplanar la salida para pasarla a capas más densas
    #Capa densa con 512 neuronas y función de activación ReLU
    #Capa de salida con una neurona de activación sigmoide para clasificar los gatos y conejos
    Flatten(),
    Dense(512, activation= "relu"),
    Dense(1, activation='sigmoid') 
])


# Compilar el modelo con optimizador Adam, pérdida binaria y métrica de precisión
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar resumen de la arquitectura del modelo
model.summary()